In [1]:
'''
Fábio Troncão
Algoritmos Genéticos
Inspirados em termos da biologia evolutiva, tais como hereditariedade, seleção natural, cruzamento e mutação,
os Algoritmos Genéticos são uma técnica de Computação centrada na busca de soluções para problemas de otimização.

A presente codificação foi baseada na biblioteca "deap".

Abaixo se pode observar a importação dos pacotes e módulos necessários.
'''

# 1- Importa se os pacotes e módulos

import random
import numpy

#!pip install deap # Instalação de DEAP
#DEAP documentation e exemplos de algoritmos no site: https://deap.readthedocs.io/en/master/

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

In [2]:
'''
Seleção e Formulação do Problema
Os Algortimos Genéticos buscam por soluções próximas do ótimo. Logo, são empregados a problemas 
para os quais não existem algoritmos conhecidos que encontrem a solução ótima em tempo polinomial.

Dessa forma, nada mais natural do que escolher um desses problemas para aplicar um algoritmo genético. 
Um exemplo clássico é o Problema do "Caixeiro Viajante".

O Problema do Caixeiro Viajante, reside no objetivo de encontrar a menor rota possível para visitar um conjunto de cidades 
e retornar à origem.

O trecho de código abaixo gera um grafo para o problema do caixeiro viajante.
O grafo é gerado em uma matriz bidimensional, sendo as distâncias valores inteiros aleatórios no intervalo [10, 100].
'''

# 2- Geração do grafo para o problema do caixeiro

# função graphTSP(numCities, minDist, maxDist)
# parâmetros: 
#   numCities: número de cities
#   minDist: menor valor de distância
#   maxDist: maior valor de distância
# retorno:
#   cities: grafo de cidades. As distância 
#   entre duas cidades são determinadas aleatoriamente entre minDist e maxDist

def graphTSP(numCities, minDist, maxDist):
  cities = numpy.zeros((numCities, numCities), dtype = int)
  for i in range(numCities):
    for j in range(numCities):
      if (j>i):
        cities[i, j] = random.randint(minDist, maxDist)
      elif (j<i):
        cities[i, j] = cities[j, i]
  return cities

numCities = 5     #  Número de cidade inicial

while(True):
  numCities = int(input('Digite o número de cidades: '))
  if (numCities > 4):
    break
  else:
    print('O número de cidades deve ser maior que 4!')

cities = graphTSP(numCities, 10, 100) # as distancias deixei fixo na chamada para numeros aleatorios de 10 a 100
print('Grafo:\n', cities)

Digite o número de cidades: 10
Grafo:
 [[  0  86  59  47  20  78  22  71  30  17]
 [ 86   0  13  12  91  97  32  12  11  73]
 [ 59  13   0  39  19  14  74  87  27  74]
 [ 47  12  39   0  60  89  95  12  10  99]
 [ 20  91  19  60   0  13  52  36  30  63]
 [ 78  97  14  89  13   0 100  28  17  33]
 [ 22  32  74  95  52 100   0  29  95  11]
 [ 71  12  87  12  36  28  29   0  72  80]
 [ 30  11  27  10  30  17  95  72   0  81]
 [ 17  73  74  99  63  33  11  80  81   0]]


In [3]:
'''
População e Indivíduos
A população é o conjunto de indivíduos que estão sendo cogitados como solução e que serão usados para criar 
o novo conjunto de indivíduos para análise.

O indivíduo, também chamado de cromossomo ou string, é uma possível solução para um dado problema. 
Cada indivíduo é um conjunto de parâmetros (genes), cuja representação depende do domínio do problema.

Genótipo é a sequência de genes. No caso do problema do caixeiro, cada gene é uma cidade (número do vértice).
EX: [0, 2, 1, 4, 3] é um genótipo de indivíduo para o problema do caixeiro com 5 cidades.
Fenótico é o produto da interação de todos os genes.
Para o caixeiro seria a rota.
'''

# 3- Definição da Geração dos Indivíduos (Defenido onde a população e os individuos vao ser gerados)

# Usar o FitnessMin com o peso -1 para minimizar a soluçao (ja que quero encontrar o minimo)
creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) # minimizar = peso negativo 
# meu individuo vai ser uma lista.
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
# gerador de parâmetros (atributos inteiros gerados automatcamente)
toolbox.register("attr_int", random.randint, 0, numCities-1)

# define como os indivíduos/população é gerada
# O individuo vai ser criado de uma lista de individuos (que é uma lista de caracteristicas de cidades
#cada uma dessas caracteristicas é um atributo inteiro que é gerado aleatorio entre 0 e numCities -1 e a quantidade
#de caracteristicas que eu tenho ou seja de genes é numCities)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, numCities)
#Geraçao da população formada plos individuos
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [4]:
'''
Avaliação da Aptidão
A Função de Aptidão ou "Fitness" mede o grau de aptidão de cada indivíduo da população.

O grau de aptidão é a qualidade da solução (indivíduo) frente ao problema, 
ou seja, o quão próximo um indivíduo está da solução desejada ou quão boa é esta solução.

Para o Problema do Caixeiro Viajante a aptidão está associada a menor rota. 
Assim, abaixo temos a função que mede o custo de uma rota (indivíduo).
'''
# 4- Função para a avaliação da aptidão

# Função evalRoute(individual)
# parâmetros:
#   individual: recebe uma rota
# retorno:
#   (cost, ): tupla contendo apenas o custo da rota (cost).
# * precisa ser uma tupla devido a exigências do pacote DEAP 
def evalRoute(individual):
  cost = 0
# aqui vai calculando, ou seja somando a distancia entre as cidades com base do individuo.
  for i in range(1, len(individual)):
    if (individual.count(individual[i])>1):
      cost = cost + 1000000 # Ao encontrar cidde repetida tem penalidade de 1 milhao por repetir cidade
    cost = cost + cities[individual[i-1], individual[i]]        
  cost = cost + cities[individual[i],individual[0]]
  return (cost,) # Retorna uma tupla com o custo (Tupla porcausa do padrao da biblioteca)

In [5]:
'''
Processamento do Algoritmo Genético

Gerações
    A cada iteração do algoritmo, um novo conjunto de indivíduos é gerado a partir da população anterior.
    Cada novo conjunto é chamado de “Geração”.
    Através da criação de uma grande quantidade de gerações que é possível obter resultados dos Algoritmos Genéticos.

Avaliação de Aptidão:
    A função de aptidão é aplicada ao fenótipo do indivíduo.

Seleção
    Selecionar os indivíduos sobre os quais serão aplicados os operadores genéticos.
    Escolhe preferencialmente, embora não exclusivamente, indivíduos com maior aptidão.
    Há diversas técnicas de seleção, entre elas há o método de seleção por Roleta e o método de seleção por Torneio.

Cruzamento
    Também conhecida por Crossover ou Recombinação.
    Recombinação de características dos pais.
        Permite que as próximas gerações herdem essas características.
    Escolhe dois indivíduos e troca trechos dos cromossomos entre eles.
    
Mutação
    Introdução e manutenção da diversidade genética.
    Altera aleatoriamente um ou mais genes no cromossomo.
    
    [{0},1,3,4,2] ---- Mutação --> [{4},1,3,4,2]  O 0 sofreu mutação eleatoriamente pelo 4
    (Que no caso gerou um individuo que não é soluçao je que o 4 se repete)
'''

# 5- Processamento do Algoritmo Genético

# Definindo avaliação de aptidão, seleção, cruzamento e mutação 
toolbox.register("evaluate", evalRoute) # Na avalição irei usar a evalRout
toolbox.register("select", tools.selTournament, tournsize=3) # seleção por torneio, onde esão 3 individuos por torneio.
toolbox.register("mate", tools.cxOnePoint) # um ponto de cruzamento
toolbox.register("mutate", tools.mutUniformInt, low=0, up=numCities-1, indpb=0.05)#Mutação (valor int entre 0 e numCities -1)

def main():
  print('Execução do algoritmo genético:')

  # random.seed(64)
  NGEN = 100     # número de gerações
  MU = 50        # tamanho da população
  LAMBDA = 100   # número de filhos gerados
  CXPB = 0.7     # probabilidade de cruzamento
  MUTPB = 0.3    # probabilidade de mutação
  
  pop = toolbox.population(n=MU) # Gero minha populaçao inicial
  hof = tools.ParetoFront()
  stats = tools.Statistics(lambda ind: ind.fitness.values)# gera as estatisticas durante a execuçao do programa.
  stats.register("avg", numpy.mean, axis=0)
  stats.register("std", numpy.std, axis=0)
  stats.register("min", numpy.min, axis=0)
  stats.register("max", numpy.max, axis=0)
  
  algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                            halloffame=hof) #(E aqui eu estou chamando a execuçao do algoritmo genetico, duante o numero de
#gerações que no caso sao 100)
      
  print('\nRota:', hof[0],'\nCusto:', evalRoute(hof[0])[0])# e no final irei mostrar a rota e o custo da rota
  return pop, stats, hof
                 
if __name__ == "__main__":
    main()

#Legenda:
#gen -> numero de geração
#nevals -> numero de elementos que foram avaliados
#avg -> valor medio das soluçoes
#std -> desvio padrão
#min -> valor minimo
#max -> valor maximo

# no caso o que mais me interessa é o minimo, ja que minha intenção é minimizar o custo da rota.
# Pode se observar entao que ao longo das geraçoes o custo da rota ele vai decaindo, porque quanto menor o custo
# maior é a aptidão do individuo

Execução do algoritmo genético:
gen	nevals	avg        	std               	min       	max       
0  	50    	[5500406.9]	[1577948.39840088]	[2000395.]	[8000504.]
1  	100   	[3980428.84]	[948443.20779056] 	[2000486.]	[6000322.]
2  	100   	[3300426.82]	[854383.8431495]  	[2000359.]	[6000479.]
3  	100   	[2580416.54]	[776913.95705275] 	[1000367.]	[4000499.]
4  	100   	[2280399.86]	[749387.83070133] 	[1000367.]	[3000417.]
5  	100   	[1880384.18]	[765245.95098793] 	[1000335.]	[3000408.]
6  	100   	[1520387.04]	[670536.50198169] 	[1000335.]	[3000521.]
7  	100   	[1240380.38]	[471598.91782221] 	[1000335.]	[3000380.]
8  	100   	[1040372.58]	[397987.73205721] 	[396.]    	[2000504.]
9  	100   	[920362.24] 	[337010.50456284] 	[479.]    	[2000348.]
10 	100   	[720375.86] 	[491474.27727644] 	[396.]    	[2000348.]
11 	100   	[300425.6]  	[499948.8813286]  	[375.]    	[2000364.]
12 	100   	[20430.4]   	[139986.3795483]  	[375.]    	[1000335.]
13 	100   	[409.6]     	[42.1160302]      	[375.]    	[479.]